In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [4]:
dataset = spark.read.csv('Dataset/seeds_dataset.csv', inferSchema=True, header=True)

In [5]:
dataset.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [6]:
dataset.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22)]

In [8]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler

In [9]:
dataset.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [10]:
assembler = VectorAssembler(inputCols=dataset.columns, outputCol='features')

In [11]:
final_data = assembler.transform(dataset)

In [12]:
final_data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



In [13]:
from pyspark.ml.feature import StandardScaler

In [14]:
scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures')

In [15]:
scaler_model = scaler.fit(final_data)

In [16]:
final_data = scaler_model.transform(final_data)

In [17]:
final_data.head(1)

[Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), scaledFeatures=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))]

In [18]:
kmeans = KMeans(featuresCol='scaledFeatures', k=3)

In [19]:
model = kmeans.fit(final_data)

In [21]:
print('WSSSE')
print(model.computeCost(final_data))

WSSSE


AttributeError: 'KMeansModel' object has no attribute 'computeCost'

In [22]:
centers = model.clusterCenters()

In [23]:
print(centers)

[array([ 6.35645488, 12.40730852, 37.41990178, 13.93860446,  9.7892399 ,
        2.41585013, 12.29286107]), array([ 4.06660859, 10.14191893, 35.84098009, 11.81592066,  7.52397236,
        3.1823335 , 10.39801233]), array([ 4.93382436, 10.94691274, 37.30542404, 12.41332714,  8.60366812,
        1.82917353, 10.40106154])]


In [24]:
model.transform(final_data).select('prediction').show()

+----------+
|prediction|
+----------+
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         0|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         1|
+----------+
only showing top 20 rows

